In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy, scipy as sklearn, librosa, urllib
import librosa.display
from IPython.display import Audio
import json 
import seaborn as sns
from sklearn.cluster import KMeans
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
import keras
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from os import path

from itertools import cycle
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, silhouette_score,roc_auc_score, precision_recall_fscore_support
from tqdm import tqdm
import opensmile


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import regularizers
from joblib import dump, load

from tensorflow.keras.datasets import mnist


In [2]:
smile = opensmile.Smile(
feature_set=opensmile.FeatureSet.eGeMAPSv02,
feature_level=opensmile.FeatureLevel.Functionals,
)

smile.feature_names

['F0semitoneFrom27.5Hz_sma3nz_amean',
 'F0semitoneFrom27.5Hz_sma3nz_stddevNorm',
 'F0semitoneFrom27.5Hz_sma3nz_percentile20.0',
 'F0semitoneFrom27.5Hz_sma3nz_percentile50.0',
 'F0semitoneFrom27.5Hz_sma3nz_percentile80.0',
 'F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2',
 'F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope',
 'loudness_sma3_amean',
 'loudness_sma3_stddevNorm',
 'loudness_sma3_percentile20.0',
 'loudness_sma3_percentile50.0',
 'loudness_sma3_percentile80.0',
 'loudness_sma3_pctlrange0-2',
 'loudness_sma3_meanRisingSlope',
 'loudness_sma3_stddevRisingSlope',
 'loudness_sma3_meanFallingSlope',
 'loudness_sma3_stddevFallingSlope',
 'spectralFlux_sma3_amean',
 'spectralFlux_sma3_stddevNorm',
 'mfcc1_sma3_amean',
 'mfcc1_sma3_stddevNorm',
 'mfcc2_sma3_amean',
 'mfcc2_sma3_stddevNorm',
 'mfcc3_sma3_amean',
 'mfcc3_sma3_stddevNorm',
 'mfcc4_

In [3]:
def return_features(file,pad):
    X, sample_rate = librosa.load(file)
    max_ = X.shape[0] / sample_rate
    # if max_ < pad:
    #     length = (pad * sample_rate) -  (X.shape[0] / sample_rate)
    #     X = librosa.util.pad_center(X, size = length, mode = 'constant')
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
    )
    y = smile.process_signal(X,sample_rate)
    #  smile.process_file(file, end=3)
    return y
def get_max_min(files):
    min_, max_ = 100, 0
    for file in files:
        sound_file, samplerate = librosa.load(file)
        t = sound_file.shape[0] / samplerate
        if t < min_:
            min_ = t
        if t >= max_:
            max_ = t

    return np.round(max_,2) + 0.01, min_

In [4]:
#add white noise to the original signal
def noise_addition(data,noise_percentage_factor=0.035):
    noise = np.random.normal(0, data.std(), data.size)
    augmented_data = data + noise * noise_percentage_factor
    return augmented_data

#lower the pitch of the original signal
def pitch_scaling(data, sr, num_semitones=-2):
    return librosa.effects.pitch_shift(y = data,sr = sr,n_steps = num_semitones)

#increase the pitch of the original signal
def pitch_scaling2(data, sr, num_semitones=2):
    return librosa.effects.pitch_shift(y = data,sr = sr,n_steps = num_semitones)



In [9]:
data_df = pd.read_csv("Datasets/EMOVO/data.csv")

data_df = pd.read_csv(("Datasets/emozionalmente/metadata/samples.csv"))

In [8]:
data_df

,file_name,label,actor
0,f1/dis-f1-b1.wav,disgust,f1
1,f1/dis-f1-b2.wav,disgust,f1
2,f1/dis-f1-b3.wav,disgust,f1
3,f1/dis-f1-d1.wav,disgust,f1
4,f1/dis-f1-d2.wav,disgust,f1
...,...,...,...
583,m3/tri-m3-n1.wav,sadness,m3
584,m3/tri-m3-n2.wav,sadness,m3
585,m3/tri-m3-n3.wav,sadness,m3
586,m3/tri-m3-n4.wav,sadness,m3


In [11]:
data_df.rename(columns={'emotion_expressed': 'label'}, inplace=True)

In [13]:
data_df

,file_name,sentence,sentence_language,actor,label
0,1614951631538.wav,E’ andato a scuola dopo pranzo,italian,d4bbc5043503b9aed309ede00854ee48937684b57a1cc0...,anger
1,1613671614352.wav,Tornerà a casa presto,italian,b830cbf6e79aeeeec1fbf2ef3be741628ffc574c9b15f3...,anger
2,1613994293936.wav,Vado in biblioteca,italian,acdd987f9c1700b25898d3bd30c201df5ad4f34b3ca5eb...,anger
3,1617135480050.wav,E’ andato a scuola dopo pranzo,italian,28043d2516f2d956b81ce66cc01fbd427ac54ff1eb3a07...,anger
4,1613658275427.wav,Zia Marta ha detto che devo stare a casa stasera,italian,5b8476bb94a4a1a17df32b417ec035029f48e4487c9055...,anger
...,...,...,...,...,...
6897,1613310701161.wav,E’ andato a scuola dopo pranzo,italian,1ae2a40d6e3a1b8c6041da953f650dc017fea8c646648b...,surprise
6898,1613061195435.wav,E’ impegnato in una riunione,italian,f4d0c9178a221556efe45a4ea3d2bb77b470626409af7f...,surprise
6899,1615231434303.wav,L’ho incontrato oggi dopo 2 anni,italian,7ce215563f7b620fdd32c6618a95215100b4fbede0eb4f...,surprise
6900,1613403675624.wav,E’ una notte stellata,italian,1dab1d691c3cb60f475163ff79b4fa2f58bac807098245...,surprise


### CLEAR

In [17]:
data_df["file_name"][0]

'1614951631538.wav'

In [18]:


X, sample_rate = librosa.load("Datasets/emozionalmente/audio/1614951631538.wav")

In [19]:
train_data = pd.DataFrame(columns=['filename', 'features', 'label'])
max, min = get_max_min('Datasets/emozionalmente/audio/'+data_df.file_name)
filenames= data_df.file_name
labels= data_df.label

tot = range(list(data_df.index)[-1])



for index,file in tqdm(zip(tot,filenames)):
    train_data.loc[index] = [file, return_features('Datasets/emozionalmente/audio/'+file,max), labels[index]]


6901it [09:02, 12.72it/s]


In [20]:
train_data.to_pickle("emozionalmente_clear.pkl")

#### PITCH ONLY

In [2]:
def return_features(file,pad):
    X, sample_rate = librosa.load(file)
    max_ = X.shape[0] / sample_rate
    # if max_ < pad:
    #     length = (pad * sample_rate) -  (X.shape[0] / sample_rate)
    #     X = librosa.util.pad_center(X, size = length, mode = 'constant')
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
    )

    X = pitch_scaling(X, sample_rate)
    y = smile.process_signal(X,sample_rate)
    #  smile.process_file(file, end=3)
    return y

def return_features2(file,pad):
    X, sample_rate = librosa.load(file)
    max_ = X.shape[0] / sample_rate
    # if max_ < pad:
    #     length = (pad * sample_rate) -  (X.shape[0] / sample_rate)
    #     X = librosa.util.pad_center(X, size = length, mode = 'constant')
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
    )

    X = pitch_scaling2(X, sample_rate)
    y = smile.process_signal(X,sample_rate)
    #  smile.process_file(file, end=3)
    return y

def get_max_min(files):
    min_, max_ = 100, 0
    for file in files:
        sound_file, samplerate = librosa.load(file)
        t = sound_file.shape[0] / samplerate
        if t < min_:
            min_ = t
        if t >= max_:
            max_ = t

    return np.round(max_,2) + 0.01, min_

In [3]:
data_df = pd.read_csv("EMOVO_dataset/data.csv")

In [6]:
train_data = pd.DataFrame(columns=['filename', 'features', 'label'])
max, min = get_max_min('EMOVO_dataset/'+data_df.file_name)
filenames= pd.concat([data_df.file_name]*2, ignore_index=True) 
labels= pd.concat([data_df.label]*2, ignore_index=True) 
features = []
tot = range(list(data_df.index)[-1]*2)
tot_m = (list(data_df.index)[-1]*2)

for index,file in tqdm(zip(tot,filenames)):
    if index <= tot_m//2:
        train_data.loc[index] = [file, return_features('EMOVO_dataset/'+file,max), labels[index]]
    else:
        train_data.loc[index] = [file, return_features2('EMOVO_dataset/'+file,max), labels[index]]

1174it [01:38, 11.93it/s]


In [7]:
train_data.to_pickle("pitch_functional_emovo.pkl")